# Notebook to Python Script Converter

Convert Jupyter notebooks to executable Python scripts.

## Features:
- Convert single notebook
- Batch convert multiple notebooks
- Clean output (remove magic commands)
- Add script header

## Configuration

In [1]:
# Import Setup
import sys
from pathlib import Path

# Add parent directory to Python path to import modules
repo_root = Path.cwd().parent
# No longer needed - using installed package

print(f"✅ Python path configured")
print(f"   Repository root: {repo_root}")

✅ Python path configured
   Repository root: /home/fabric/work/fabric-generic-cluster


In [2]:
# Configuration
# Notebook files are in notebooks/ directory
NOTEBOOK_DIR = Path.cwd()

# Output directory for converted scripts
SCRIPTS_OUTPUT_DIR = repo_root / "scripts"
SCRIPTS_OUTPUT_DIR.mkdir(exist_ok=True)

# Configuration for conversion
notebook_filename = NOTEBOOK_DIR / "notebook-create-slice.ipynb"
output_filename = SCRIPTS_OUTPUT_DIR / "notebook-create-slice.py"

# Options
remove_magic_commands = True  # Remove IPython magic commands (%, %%)
add_header = True             # Add script header with metadata
include_markdown = False      # Include markdown as comments

print(f"✅ Notebook directory: {NOTEBOOK_DIR}")
print(f"✅ Scripts output directory: {SCRIPTS_OUTPUT_DIR}")

✅ Notebook directory: /home/fabric/work/fabric-generic-cluster/notebooks
✅ Scripts output directory: /home/fabric/work/fabric-generic-cluster/scripts


## Method 1: Convert Single Notebook (Enhanced)

In [3]:
import nbformat
from nbconvert import PythonExporter
from datetime import datetime
import re

def convert_notebook_to_python(
    notebook_path,
    output_path,
    remove_magic=True,
    add_header=True,
    include_markdown=False
):
    """
    Convert Jupyter notebook to Python script with options.
    
    Args:
        notebook_path: Path to input notebook
        output_path: Path to output Python file
        remove_magic: Remove IPython magic commands
        add_header: Add script header
        include_markdown: Include markdown cells as comments
    """
    print(f"Converting: {notebook_path}")
    
    # Read notebook
    with open(notebook_path, 'r') as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Export to Python
    exporter = PythonExporter()
    if not include_markdown:
        exporter.exclude_markdown = True
    
    source, _ = exporter.from_notebook_node(notebook)
    
    # Post-process
    if remove_magic:
        # Remove IPython magic commands
        lines = source.split('\n')
        cleaned_lines = []
        for line in lines:
            # Skip lines starting with % or %%
            if not line.strip().startswith(('get_ipython()', '%', '%%')):
                cleaned_lines.append(line)
        source = '\n'.join(cleaned_lines)
    
    # Add header
    if add_header:
        header = f'''#!/usr/bin/env python3
"""
Auto-generated from: {notebook_path}
Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

FABRIC Slice Management Framework
"""

'''
        source = header + source
    
    # Write output
    with open(output_path, 'w') as f:
        f.write(source)
    
    print(f"✅ Converted to: {output_path}")
    print(f"   Lines: {len(source.split(chr(10)))}")

# Convert
convert_notebook_to_python(
    notebook_filename,
    output_filename,
    remove_magic=remove_magic_commands,
    add_header=add_header,
    include_markdown=include_markdown
)

Converting: /home/fabric/work/fabric-generic-cluster/notebooks/notebook-create-slice.ipynb
✅ Converted to: /home/fabric/work/fabric-generic-cluster/scripts/notebook-create-slice.py
   Lines: 358


## Method 2: Using nbconvert CLI

Quick command-line conversion:

In [ ]:
# Simple conversion
!jupyter nbconvert --to script {notebook_filename}

# With custom output name
# !jupyter nbconvert --to script --output {output_filename} {notebook_filename}

## Method 3: Batch Conversion

Convert multiple notebooks at once:

In [ ]:
from pathlib import Path

def batch_convert_notebooks(
    pattern="notebook-*.ipynb",
    output_dir="scripts",
    **kwargs
):
    """
    Batch convert notebooks matching a pattern.
    
    Args:
        pattern: Glob pattern for notebook files
        output_dir: Directory for output scripts
        **kwargs: Options passed to convert_notebook_to_python
    """
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # Find notebooks
    notebooks = list(Path('.').glob(pattern))
    
    if not notebooks:
        print(f"⚠️  No notebooks found matching: {pattern}")
        return
    
    print(f"Found {len(notebooks)} notebook(s) to convert:\n")
    
    # Convert each
    for nb_path in notebooks:
        output_file = output_path / f"{nb_path.stem}.py"
        try:
            convert_notebook_to_python(
                str(nb_path),
                str(output_file),
                **kwargs
            )
        except Exception as e:
            print(f"❌ Error converting {nb_path}: {e}")
    
    print(f"\n✅ Batch conversion complete!")
    print(f"   Output directory: {output_dir}")

# Example: Convert all notebook-*.ipynb files
# batch_convert_notebooks(
#     pattern="notebook-*.ipynb",
#     output_dir="scripts",
#     remove_magic=True,
#     add_header=True
# )

## Method 4: Convert Specific Notebooks

Convert a predefined list of notebooks:

In [ ]:
# List of notebooks to convert
notebooks_to_convert = [
    ("notebook-1.ipynb", "scripts/deploy_slice.py"),
    ("notebook-aux-setup-environment.ipynb", "scripts/setup_environment.py"),
    # Add more as needed
]

print("Converting notebooks...\n")

for nb_in, py_out in notebooks_to_convert:
    try:
        convert_notebook_to_python(
            nb_in,
            py_out,
            remove_magic=True,
            add_header=True
        )
        print()
    except FileNotFoundError:
        print(f"⚠️  Skipping {nb_in} (not found)\n")
    except Exception as e:
        print(f"❌ Error with {nb_in}: {e}\n")

print("✅ Conversion complete!")

## Utility: Compare Notebook vs Script

Verify conversion:

In [ ]:
def compare_notebook_and_script(notebook_path, script_path):
    """
    Compare notebook and generated script.
    """
    from pathlib import Path
    
    nb_path = Path(notebook_path)
    py_path = Path(script_path)
    
    if not nb_path.exists():
        print(f"❌ Notebook not found: {notebook_path}")
        return
    
    if not py_path.exists():
        print(f"❌ Script not found: {script_path}")
        return
    
    # Count lines
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)
        nb_cells = len(nb.cells)
        nb_code_cells = sum(1 for cell in nb.cells if cell.cell_type == 'code')
    
    with open(script_path) as f:
        py_lines = len(f.readlines())
    
    print(f"📊 Comparison:")
    print(f"   Notebook: {notebook_path}")
    print(f"     • Total cells: {nb_cells}")
    print(f"     • Code cells: {nb_code_cells}")
    print(f"   Script: {script_path}")
    print(f"     • Lines: {py_lines}")

# Example
# compare_notebook_and_script(notebook_filename, output_filename)

## Summary

This notebook provides multiple methods for converting Jupyter notebooks to Python scripts:

1. **Single conversion** - Convert one notebook with options
2. **CLI conversion** - Quick command-line conversion
3. **Batch conversion** - Convert multiple notebooks matching a pattern
4. **Targeted conversion** - Convert specific notebooks from a list

### Options Available:
- ✅ Remove IPython magic commands
- ✅ Add script header with metadata
- ✅ Include/exclude markdown cells
- ✅ Custom output paths

### Use Cases:
- **Automation**: Convert notebooks to Python scripts for CI/CD
- **Deployment**: Create standalone scripts from notebooks
- **Testing**: Run notebook code in non-interactive environments